In [1]:
import numpy as np
import pandas as pd

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('../data/ift6758_dfwseasons 3/Allseasons.csv')

# EDA

In [3]:
df.head()

,eventType,period,periodTime,periodType,gameID,teamOfShooter,homeOrAway,xCoord,yCoord,shooter,goalie,shotType,emptyNet,strength,season,rinkSide
0,Shot,1,01:11,REGULAR,2016020001,Toronto Maple Leafs,away,-77.0,5.0,Mitchell Marner,Craig Anderson,Wrist Shot,NaN,NaN,2016,right
1,Shot,1,02:53,REGULAR,2016020001,Ottawa Senators,home,86.0,13.0,Chris Kelly,Frederik Andersen,Wrist Shot,NaN,NaN,2016,left
2,Shot,1,04:01,REGULAR,2016020001,Ottawa Senators,home,23.0,-38.0,Cody Ceci,Frederik Andersen,Wrist Shot,NaN,NaN,2016,left
3,Shot,1,04:46,REGULAR,2016020001,Ottawa Senators,home,33.0,-15.0,Erik Karlsson,Frederik Andersen,Slap Shot,NaN,NaN,2016,left
4,Shot,1,06:46,REGULAR,2016020001,Toronto Maple Leafs,away,-34.0,28.0,Martin Marincin,Craig Anderson,Wrist Shot,NaN,NaN,2016,right


In [4]:
games = df.groupby(['season', 'periodType'])['gameID'].unique()

In [5]:
games.index

MultiIndex([(2016, 'OVERTIME'),
            (2016,  'REGULAR'),
            (2016, 'SHOOTOUT'),
            (2017, 'OVERTIME'),
            (2017,  'REGULAR'),
            (2017, 'SHOOTOUT'),
            (2018, 'OVERTIME'),
            (2018,  'REGULAR'),
            (2018, 'SHOOTOUT'),
            (2019, 'OVERTIME'),
            (2019,  'REGULAR'),
            (2019, 'SHOOTOUT'),
            (2020, 'OVERTIME'),
            (2020,  'REGULAR'),
            (2020, 'SHOOTOUT')],
           names=['season', 'periodType'])

### Shots across the middle red line

In [6]:
df[((df['rinkSide'] == 'left') & (df['xCoord'] < 0)) | ((df['rinkSide'] == 'right') & (df['xCoord'] > 0))].groupby('season')['eventType'].value_counts()

season  eventType
2016    Shot          946
        Goal          125
2017    Shot         2137
        Goal          129
2018    Shot         2374
        Goal          150
2019    Shot         4278
        Goal          374
2020    Shot         1329
        Goal          110
Name: eventType, dtype: int64

In [7]:
def normalize_df(df):
    df_normalized = df.copy()
    df_normalized['xCoordNorm'] = np.where(df['rinkSide'] == 'left', df['xCoord'], -1 * df['xCoord'])
    df_normalized['yCoordNorm'] = np.where(df['rinkSide'] == 'left', df['yCoord'], -1 * df['yCoord'])

    return df_normalized

In [8]:
df_normalized = normalize_df(df)

In [9]:
def plot_rink_overlay(data, normalize=False, title='', marker='o', size=100, color='#ea6969', alpha=.7):
    fig, ax = plt.subplots(figsize=(13,8.5))
    plt.axis('on')

    img = plt.imread('../figures/nhl_rink.png')
    # rink dimensions
    ext = [-100, 100, -42.5, 42.5]
    plt.imshow(img, zorder=0, extent=ext, aspect=1)

    # plot the points on top of the image
    xLabel, yLabel = ('xCoordNorm', 'yCoordNorm') if normalize else ('xCoord', 'yCoord')
    plt.scatter(data[xLabel], data[yLabel], s=size, c=color, alpha=alpha, marker=marker, label='shot')

    # plot the opposing team's net
    net_x, net_y = (89, 0) if (data['rinkSide'] == 'left') or normalize else (-89, 0)
    plt.scatter(net_x, net_y, s=200, c='blue', alpha=alpha, marker='x', label='net')
    plt.title(title)
    plt.legend()


@interact(
    season_year=widgets.IntSlider(min=2016, max=2020, step=1),
    season_type=['REGULAR', 'OVERTIME', 'SHOOTOUT']
)
def show_data(season_year, season_type):
    season_games = games[(season_year, season_type)]

    @interact(game_id=widgets.SelectionSlider(options=season_games))
    def show_game(game_id):
        selected_game = df_normalized[df_normalized['gameID'] == game_id]

        @interact(event_id=widgets.IntSlider(min=0, max=selected_game.shape[0]-1, step=1, value=0))
        def show_event(event_id, normalize_shots=True):
            event = selected_game.iloc[event_id]
            title = f"{event['eventType']} \t {event['shotType']} \t {event['shooter']} -> {event['goalie']}"
            plot_rink_overlay(event, normalize=normalize_shots, title=title, marker='o', size=300, color='red', alpha=1)
            print(event)


interactive(children=(IntSlider(value=2016, description='season_year', max=2020, min=2016), Dropdown(descripti…